Copyright (c) Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Use FLAML to Tune OpenAI Models

In this notebook, we tune OpenAI models for code generation. We use [the HumanEval benchmark](https://huggingface.co/datasets/openai_humaneval) released by OpenAI for synthesizing programs from docstrings. 

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [openai] option:
```bash
pip install flaml[openai]
```

In [1]:
# %pip install flaml[openai] datasets

Set your OpenAI key:

In [2]:
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "<your OpenAI API key here>"

## Load dataset

First, we load the humaneval dataset. The dataset contains 164 examples. We use the first 20 for tuning the generation hyperparameters and the remaining for evaluation. In each example, the "prompt" is the prompt string for eliciting the code generation, "test" is the Python code for unit test for the example, and "entry_point" is the function name to be tested.

In [3]:
import datasets

seed = 41
data = datasets.load_dataset("openai_humaneval")["test"].shuffle(seed=seed)
n_tune_data = 20
tune_data = [
    {
        "prompt": data[x]["prompt"],
        "test": data[x]["test"],
        "entry_point": data[x]["entry_point"],
    }
    for x in range(n_tune_data)
]
test_data = [
    {
        "prompt": data[x]["prompt"],
        "test": data[x]["test"],
        "entry_point": data[x]["entry_point"],
    }
    for x in range(n_tune_data, len(data))
]


Found cached dataset openai_humaneval (/home/vscode/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/vscode/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75/cache-1e8448101c1b32e8.arrow


Check a tuning example:

In [4]:
print(tune_data[1]["prompt"])


def compare(game,guess):
    """I think we all remember that feeling when the result of some long-awaited
    event is finally known. The feelings and thoughts you have at that moment are
    definitely worth noting down and comparing.
    Your task is to determine if a person correctly guessed the results of a number of matches.
    You are given two arrays of scores and guesses of equal length, where each index shows a match. 
    Return an array of the same length denoting how far off each guess was. If they have guessed correctly,
    the value is 0, and if not, the value is the absolute difference between the guess and the score.
    
    
    example:

    compare([1,2,3,4,5,1],[1,2,3,4,2,-2]) -> [0,0,0,0,3,3]
    compare([0,5,0,0,0,4],[4,1,1,0,0,-2]) -> [4,4,1,0,0,6]
    """



Here is one example of the unit test code for verifying the correctness of the generated code:

In [5]:
print(tune_data[1]["test"])

def check(candidate):

    # Check some simple cases
    assert candidate([1,2,3,4,5,1],[1,2,3,4,2,-2])==[0,0,0,0,3,3], "This prints if this assert fails 1 (good for debugging!)"
    assert candidate([0,0,0,0,0,0],[0,0,0,0,0,0])==[0,0,0,0,0,0], "This prints if this assert fails 1 (good for debugging!)"
    assert candidate([1,2,3],[-1,-2,-3])==[2,4,6], "This prints if this assert fails 1 (good for debugging!)"
    assert candidate([1,2,3,5],[-1,2,3,4])==[2,0,0,1], "This prints if this assert fails 1 (good for debugging!)"

    # Check some edge cases that are easy to work out by hand.
    assert True, "This prints if this assert fails 2 (also good for debugging!)"




## Define Success Metric

Before we start tuning, we need to define the success metric we want to opotimize. For each code generation task, if one of the returned responses can pass the test, we consider the task as successfully solved. Then we can define the mean success rate of a collection of tasks.

### Define a code executor

First, we write a simple code executor. The code executor takes the generated code and the test code as the input, and execute them with a timer.

In [6]:
import signal
import subprocess
import sys

def timeout_handler(signum, frame):
    raise TimeoutError("Timed out!")

signal.signal(signal.SIGALRM, timeout_handler)
max_exec_time = 3  # seconds

def execute_code(code):
    code = code.strip()
    with open("codetest.py", "w") as fout:
        fout.write(code)
    try:
        signal.alarm(max_exec_time)
        result = subprocess.run(
            [sys.executable, "codetest.py"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
        )
        signal.alarm(0)
    except TimeoutError:
        return 0
    return int(result.returncode == 0)

This function will create a temp file "codetest.py" and execute it in a separate process. It allows for 3 seconds to finish that code.

### Define a function to evaluate the success for a given program synthesis task

In [7]:
def success_metrics(responses, prompt, test, entry_point):
    """Check if the task is successful.

    Args:
        responses (list): The list of responses.
        prompt (str): The input prompt.
        test (str): The test code.
        entry_point (str): The name of the function.

    Returns:
        dict: The success metrics.
    """
    success_list = []
    n = len(responses)
    for i in range(n):
        response = responses[i]
        code = f"{prompt}{response}\n{test}\ncheck({entry_point})"
        succeed = execute_code(code)
        success_list.append(succeed)
    return {
        "expected_success": 1 - pow(1 - sum(success_list) / n, n),
        "success": any(s for s in success_list),
    }


## Use the tuning data to find a good configuration

### Import the oai and tune subpackages from flaml.

FLAML has provided an API for hyperparameter optimization of OpenAI models: `oai.Completion.tune` and to make a request with the tuned config: `oai.Completion.create`. First, we import oai from flaml:

In [8]:
from flaml import oai, tune

For (local) reproducibility and cost efficiency, we cache responses from OpenAI.

In [9]:
oai.Completion.set_cache(seed)

This will create a disk cache in ".cache/{seed}". You can change `cache_path` in `set_cache()`. The cache for different seeds are stored separately.

### Perform tuning

The tuning will take a while to finish, depending on the optimization budget (~15 mins for the current budget). The tuning will be performed under the specified optimization budgets.

* inference_budget is the target average inference budget per instance in the benchmark. For example, 0.02 means the target inference budget is 0.02 dollars, which translates to 1000 tokens (input + output combined) if the Davinci model is used.
* optimization_budget is the total budget allowed to perform the tuning. For example, 5 means 5 dollars are allowed in total, which translates to 250K tokens for the Davinci model.
* num_sumples is the number of different hyperparameter configurations which is allowed to try. The tuning will stop after either num_samples trials or after optimization_budget dollars spent, whichever happens first. -1 means no hard restriction in the number of trials and the actual number is decided by optimization_budget.

Users can specify tuning data, optimization metric, optimization mode, evaluation function, search spaces etc.. The default search space is 
```python
    price1K = {
        "text-ada-001": 0.0004,
        "text-babbage-001": 0.0005,
        "text-curie-001": 0.002,
        "code-cushman-001": 0.002,  # TODO: update when available
        "code-davinci-002": 0.02,  # TODO: update when available
        "text-davinci-002": 0.02,
        "text-davinci-003": 0.02,
    }

    default_search_space = {
        "model": tune.choice(list(price1K.keys())),
        "temperature_or_top_p": tune.choice(
            [
                {"temperature": tune.uniform(0, 1)},
                {"top_p": tune.uniform(0, 1)},
            ]
        ),
        "max_tokens": tune.lograndint(50, 1000),
        "n": tune.randint(1, 100),
        "prompt": "{prompt}",
    }
```
The default search space can be overriden by users' input.
For example, the following code specifies two choices for the model, four choices for the prompt and a fixed list of stop sequences. For hyperparameters which don't appear in users' input, the default search space will be used.

In [10]:
config, analysis = oai.Completion.tune(
    data=tune_data,  # the data for tuning
    metric="expected_success",  # the metric to optimize
    mode="max",  # the optimization mode
    eval_func=success_metrics,  # the evaluation function to return the success metrics
    # log_file_name="logs/humaneval.log",  # the log file name
    inference_budget=0.02,  # the inference budget (dollar)
    optimization_budget=2,  # the optimization budget (dollar)
    # num_samples can further limit the number of trials for different hyperparameter configurations;
    # -1 means decided by the optimization budget only
    num_samples=-1,
    model=tune.choice(
        [
            # These two models are currently free to use,
            # so no actual cost will incur.
            # We use a pseudo price according to their size to simulate the case
            # where the cost is not zero.
            "code-cushman-001", 
            "code-davinci-002",
        ]
    ),
    prompt=[
        "{prompt}",
        "# Python 3{prompt}",
        "Complete the following Python function:{prompt}",
        "Complete the following Python function while including necessary import statements inside the function:{prompt}",
    ],  # the prompt templates to choose from
    stop=["\nclass", "\ndef", "\nif", "\nprint"],  # the stop sequence
)


[I 2023-02-05 17:10:50,543] A new study created in memory with name: optuna


[I 2023-02-05 17:10:50,545] A new study created in memory with name: optuna


[flaml.tune.tune: 02-05 17:10:50] {806} INFO - trial 1 config: {'model': 'code-davinci-002', 'temperature_or_top_p': {'temperature': 0.36865945026811975}, 'max_tokens': 347, 'n': 1, 'prompt': 1, 'stop': 0}


[flaml.tune.tune: 02-05 17:11:20] {215} INFO - result: {'expected_success': 0.6, 'success': 0.6, 'total_cost': 0.0925, 'cost': 0.0925, 'inference_cost': 0.004625, 'training_iteration': 0, 'config': {'model': 'code-davinci-002', 'temperature_or_top_p': {'temperature': 0.36865945026811975}, 'max_tokens': 347, 'n': 1, 'prompt': 1, 'stop': 0}, 'config/model': 'code-davinci-002', 'config/temperature_or_top_p': {'temperature': 0.36865945026811975}, 'config/max_tokens': 347, 'config/n': 1, 'config/prompt': 1, 'config/stop': 0, 'experiment_tag': 'exp', 'time_total_s': 30.013915538787842}


[flaml.tune.tune: 02-05 17:11:20] {806} INFO - trial 2 config: {'model': 'code-cushman-001', 'temperature_or_top_p': {'temperature': 0.36865945026811975}, 'max_tokens': 347, 'n': 1, 'prompt': 1, 'stop': 0}


...

[flaml.tune.tune: 02-05 17:24:07] {806} INFO - trial 22 config: {'model': 'code-cushman-001', 'max_tokens': 130, 'n': 41, 'prompt': 1, 'stop': 0, 'temperature_or_top_p': {'top_p': 0.5350214472192576}}


[flaml.tune.tune: 02-05 17:25:09] {215} INFO - result: {'expected_success': 0.6432731468677959, 'success': 0.7, 'total_cost': 1.9095459999999997, 'cost': 0.11532400000000001, 'inference_cost': 0.0057662, 'training_iteration': 0, 'config': {'model': 'code-cushman-001', 'max_tokens': 130, 'n': 41, 'prompt': 1, 'stop': 0, 'temperature_or_top_p': {'top_p': 0.5350214472192576}}, 'config/model': 'code-cushman-001', 'config/max_tokens': 130, 'config/n': 41, 'config/prompt': 1, 'config/stop': 0, 'config/temperature_or_top_p': {'top_p': 0.5350214472192576}, 'experiment_tag': 'exp', 'time_total_s': 62.217190980911255}


[flaml.tune.tune: 02-05 17:25:09] {806} INFO - trial 23 config: {'model': 'code-cushman-001', 'temperature_or_top_p': {'top_p': 0.829558896181596}, 'max_tokens': 211, 'n': 48, 'prompt': 0, 'stop': 0}


[flaml.tune.tune: 02-05 17:25:42] {215} INFO - result: {'expected_success': 0, 'total_cost': 2.00839, 'cost': 0.09884399999999999, 'training_iteration': 0, 'config': {'model': 'code-cushman-001', 'temperature_or_top_p': {'top_p': 0.829558896181596}, 'max_tokens': 211, 'n': 48, 'prompt': 0, 'stop': 0}, 'config/model': 'code-cushman-001', 'config/temperature_or_top_p': {'top_p': 0.829558896181596}, 'config/max_tokens': 211, 'config/n': 48, 'config/prompt': 0, 'config/stop': 0, 'experiment_tag': 'exp', 'time_total_s': 32.64395809173584}


[flaml.tune.tune: 02-05 17:25:42] {827} WARNING - fail to sample a trial for 100 times in a row, stopping.


### Output tuning results

After the tuning, we can print out the config and the result found by FLAML:

In [11]:
print("optimized config", config)
print("best result on tuning data", analysis.best_result)

optimized config {'model': 'code-cushman-001', 'max_tokens': 162, 'n': 44, 'prompt': '# Python 3{prompt}', 'stop': ['\nclass', '\ndef', '\nif', '\nprint'], 'top_p': 0.6715981837068888}
best result on tuning data {'expected_success': 0.7360027932558347, 'success': 0.8, 'total_cost': 1.6311499999999999, 'cost': 0.13308399999999998, 'inference_cost': 0.0064402, 'training_iteration': 0, 'config': {'model': 'code-cushman-001', 'temperature_or_top_p': {'top_p': 0.6715981837068888}, 'max_tokens': 162, 'n': 44, 'prompt': 1, 'stop': 0}, 'config/model': 'code-cushman-001', 'config/temperature_or_top_p': {'top_p': 0.6715981837068888}, 'config/max_tokens': 162, 'config/n': 44, 'config/prompt': 1, 'config/stop': 0, 'experiment_tag': 'exp', 'time_total_s': 73.76863956451416}


### Make a request with the tuned config

We can apply the tuned config on the request for an example task:

In [12]:
responses = oai.Completion.create(context=tune_data[1], **config)
print(responses)
print(success_metrics([response["text"].rstrip() for response in responses["choices"]], **tune_data[1]))


{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "    return [abs(guess[i]-game[i]) for i in range(len(game))]"
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "logprobs": null,
      "text": "    l = len(game)\n    l1 = len(guess)\n    if l!=l1:\n        return []\n    else:\n        list1 = []\n        for i in range(l):\n            if game[i]==guess[i]:\n                list1.append(0)\n            else:\n                list1.append(abs(game[i]-guess[i]))\n        return list1\n    \n    \n    \n\n# Testing"
    },
    {
      "finish_reason": "stop",
      "index": 2,
      "logprobs": null,
      "text": "    score = 0\n    res = []\n    for i in range(len(game)):\n        if game[i] == guess[i]:\n            res.append(0)\n        else:\n            res.append(abs(guess[i] - game[i]))\n    return res"
    },
    {
      "finish_reason": "stop",
      "index": 3,
      "logprobs": null,
      "text"

{'expected_success': 1.0, 'success': True}


### Evaluate the success rate on the test data

You can use flaml's `oai.Completion.eval` to evaluate the performance of an entire dataset with the tuned config. To do that you need to set `oai.Completion.data` to the data to evaluate. The following code will take a while to evaluate all the 144 test data instances. Compared to the baseline success rate (0.46) on the [HELM benchmark](https://crfm.stanford.edu/helm/latest/?group=code_humaneval), the tuned config has a success rate of 0.72. It can be further improved if the inference budget and optimization budget are further increased.

In [13]:
oai.Completion.data = test_data
result = oai.Completion.eval(analysis.best_config, prune=False, eval_only=True)
print(result)


{'expected_success': 0.6661693627028851, 'success': 0.7152777777777778, 'total_cost': 2.919666000000001, 'cost': 0.9112759999999998, 'inference_cost': 0.006328305555555556}
